In [15]:
import numpy as np
import pandas as pd
import datetime
import time
import itertools

In [ ]:
def str2time(string, time_format="%Y-%m-%d %H:%M"):
    output = datetime.datetime.strptime(string, time_format)
    return time.mktime(output.timetuple())

In [ ]:
def initialize_movie_df(movie_name, start_times, duration):
    duration_timedelta = datetime.timedelta(minutes=int(duration))
    start_timedates = [datetime.datetime.strptime(x, '%H:%M') for x in start_times]
    end_timedates = [x + duration_timedelta for x in start_timedates]
    output = pd.DataFrame({'start_times':start_timedates,\
                           'end_times':end_timedates})
    output['movie_name'] = movie_name
    return(output)

In [ ]:
"""
play around with datetime objects

str2time("2015-10-17 14:50")
datetime.timedelta(hours = 1, minutes=20) + \
datetime.datetime(year=2015,month=10,day=17)
"""

In [76]:
# make a kino program to test algorithm on

the_martian = \
initialize_movie_df('the_martian',\
                    ['13:30', '16:50', '19:30', '23:10'],\
                    duration=144)

sicario = initialize_movie_df('sicario', \
                              ['20:00', '23:10'], \
                              duration=121)

alles_steht_kopf = initialize_movie_df('alles_steht_kopf', \
                                       ['14:20', '17:00'],\
                                       95)

black_mass = initialize_movie_df('black_mass',\
                                ['19:50', '23:00'],\
                                123)

hotel_transylvania = \
initialize_movie_df('hotel_transylvania', \
                    ['14:00', '16:40'],\
                   89)

maze_runner = initialize_movie_df('maze_runner',\
                                 ['13:50'],\
                                 130)

the_intern = initialize_movie_df('the_intern',\
                                ['17:00'],\
                                121)

regression = initialize_movie_df('regression',\
                                ['22:30'],
                                107)

pan = initialize_movie_df('pan',\
                          ['14:40'],\
                          111)

everest = initialize_movie_df('everest',\
                             ['22:00'],\
                             122)

kino_program = the_martian.\
append(black_mass, ignore_index=True).\
append(hotel_transylvania, ignore_index=True).\
append(maze_runner, ignore_index=True).\
append(the_intern, ignore_index=True).\
append(regression, ignore_index=True).\
append(pan, ignore_index=True).\
append(everest, ignore_index=True).\
append(sicario, ignore_index=True).\
append(alles_steht_kopf, ignore_index=True)

kino_program.sort('start_times',\
                  ascending=True,\
                 inplace=True)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [285]:
def find_next_showings(movie_info, kino_program, waiting_time):
    movie_name = movie_info.movie_name.iloc[0]
    end_time = movie_info.end_times.iloc[0]
    max_time = end_time + \
        datetime.timedelta(minutes=waiting_time)
    row_filter = \
        (kino_program.movie_name != movie_name) & \
        (kino_program.start_times >= end_time) & \
        (kino_program.start_times <= max_time)
    return kino_program[row_filter]

def double_dip(starting_show, show_times, waiting_time):
    show_times = show_times[show_times.movie_name != starting_show.movie_name.iloc[0]]
    next_showtimes = find_next_showings(starting_show,show_times,waiting_time)
    
    node_info = starting_show.movie_name.iloc[0] + \
    " (" + starting_show.start_times.iloc[0].strftime("%H:%M") + \
    "-" + starting_show.end_times.iloc[0].strftime("%H:%M") + ")"
    
    if next_showtimes.empty:
        return node_info
    
    output = []
    
    for row in xrange(next_showtimes.shape[0]):
        next_show = next_showtimes.iloc[[row]]
        X = double_dip(next_show, show_times, waiting_time)
        if isinstance(X, basestring):
            tmp = node_info + " -> " + X
            output.append(tmp)
        else:
            if isinstance(X[0], list):
                X = [x for x in itertools.chain.from_iterable(X)]
            tmp = [node_info + " -> " + ''.join(x) for x in X]
            [output.append(x) for x in tmp]
    
    return output

In [293]:
double_dip(kino_program.iloc[6:7], kino_program, 45)

['the_martian (16:50-19:14) -> black_mass (19:50-21:53) -> everest (22:00-00:02)',
 'the_martian (16:50-19:14) -> black_mass (19:50-21:53) -> regression (22:30-00:17)']

In [225]:
kino_program

,end_times,start_times,movie_name
0,1900-01-01 15:54:00,1900-01-01 13:30:00,the_martian
8,1900-01-01 16:00:00,1900-01-01 13:50:00,maze_runner
6,1900-01-01 15:29:00,1900-01-01 14:00:00,hotel_transylvania
15,1900-01-01 15:55:00,1900-01-01 14:20:00,alles_steht_kopf
11,1900-01-01 16:31:00,1900-01-01 14:40:00,pan
7,1900-01-01 18:09:00,1900-01-01 16:40:00,hotel_transylvania
1,1900-01-01 19:14:00,1900-01-01 16:50:00,the_martian
9,1900-01-01 19:01:00,1900-01-01 17:00:00,the_intern
16,1900-01-01 18:35:00,1900-01-01 17:00:00,alles_steht_kopf
2,1900-01-01 21:54:00,1900-01-01 19:30:00,the_martian


In [203]:

starting_show = kino_program.iloc[0:1]
show_times = kino_program

next_showtimes = find_next_showings(starting_show,show_times,waiting_time)

if next_showtimes.empty:
    print starting_show.movie_name

output = []
tmp = [starting_show.movie_name]
for row in xrange(next_showtimes.shape[0]):
    next_show = next_showtimes.iloc[[row]]
    #print(next_show)
    #tmp2 = [tmp.append(x) for x in double_dip(next_show, show_times, waiting_time)]

starting_show = next_show
next_showtimes = find_next_showings(starting_show,show_times,waiting_time)

if next_showtimes.empty:
    print starting_show.movie_name.iloc[0]

#next_show.movie_name.iloc[0]

hotel_transylvania


In [153]:
"""
a -> c -> b
|    \
e     d

V = [a, b, c, d, e]
V.pop -> [a]
a.neighbors -> [c]
-- for each neighbor of a ... --

c.neighbors -> [b, d]
b.neighbors -> []
return b
d.neighbors -> []
return d
pre-pend c to returns -> [[c,b], [c,d]]

e.neighbors -> []
return e

pre-pend a to returns [[a,c,b],[a,c,d]]

def f(v):
    if neighbors.empty:
    return v
    else:
    return [v.prepend(f(x)) for x in v.neighbors]


start at a node
gather neighbors in a stack
pop neighbor in visited

"""

'\na -> c -> b\n|    e     d\n\nV = [a, b, c, d, e]\nV.pop -> [a]\na.neighbors -> [c]\n-- for each neighbor of a ... --\n\nc.neighbors -> [b, d]\nb.neighbors -> []\nreturn b\nd.neighbors -> []\nreturn d\npre-pend c to returns -> [[c,b], [c,d]]\n\ne.neighbors -> []\nreturn e\n\npre-pend a to returns [[a,c,b],[a,c,d]]\n\ndef f(v):\n    if neighbors.empty:\n    return v\n    else:\n    return [v.prepend(f(x)) for x in v.neighbors]\n\n\nstart at a node\ngather neighbors in a stack\npop neighbor in visited\n\n'

In [150]:




# remove all movies with the same name
# remove all moves outside the waiting scope
# return a dictionary with valid branches
waiting_time = 1
catch = []
for row in kino_program.iloc[[0]].itertuples():
    output = kino_program.iloc[row[0]:(row[0]+1)]
    
    new_program = find_next_showings(row[1:],\
                                    kino_program,\
                                    waiting_time)
    
    if new_program.empty:
        continue
    
    output.append(new_program.iloc[0:1], ignore_index=True)
    
    catch.append(output)
    
    """for shows in new_program.itertuples():
        find_next_showings(shows[1:],\
                           kino_program,\
                           waiting_time)
    """    
    
        
    print("****")
    print(row[3] + ": " + \
          row[2].strftime("%H:%M") + " -> " + \
          row[1].strftime("%H:%M"))
    print(output)
    print(new_program.head(0))
    #print(new_program[['movie_name', 'start_times']])
    #print(row)

#print(type(catch[0]))
print(catch[0])

****
the_martian: 13:30 -> 15:54
            end_times         start_times   movie_name
0 1900-01-01 15:54:00 1900-01-01 13:30:00  the_martian
Empty DataFrame
Columns: [end_times, start_times, movie_name]
Index: []
            end_times         start_times   movie_name
0 1900-01-01 15:54:00 1900-01-01 13:30:00  the_martian


In [151]:
pd.DataFrame.append?